<a href="https://colab.research.google.com/github/nedokormysh/kaggle_nlp2.0/blob/main/%D0%9E%D0%B1%D1%89%D0%B8%D0%B9_%D1%85%D0%BE%D0%B4_%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D1%8F_NLP2_0_%D0%B8_%D1%80%D0%B0%D0%B7%D0%BC%D1%8B%D1%88%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка библиотек

In [ ]:
!pip install optuna -q
!pip install catboost -q
!pip install natasha -q

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, Dataset, LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

import optuna
from optuna.integration import CatBoostPruningCallback

import seaborn as sns
import datetime
from wordcloud import WordCloud, STOPWORDS

import pickle

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

sns.set(style="darkgrid")
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

import gensim.downloader
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.fasttext import FastText
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

In [ ]:
# фиксируем значения
RND_STATE = 7575
N_EST = 1000
sampler = optuna.samplers.TPESampler(seed=RND_STATE)

# Этапы обработки и загрузки данных.

**ВНИМАНИЕ:** данный ноутбук составлялся для демонстрации общего хода построения моделей. В разное время использовались разные данные, и по-разному составлялись модели. Возможно сейчас запуск всех ячеек этого ноутбука вообще не даст результата. Необходимо загружать и запускать уже конкретные варианты ноутбуков. Если потребуется продемострировать именно конкретный вариант или объяснить какой-то код, то готов это сделать.

Было несколько вариантов парсинга данных. И несколько этапов. Полученные данные сохранял и загружал из гугла диска.
Какие-то последние варианты моделей и данных лежат здесь: https://drive.google.com/drive/folders/1u_2ejLOI2kMkrhoXdVYkxkKWwSQVUWlE?usp=sharing


## 1

Первый вариант был достаточно прост: у меня был уже написан парсинг для сайта фонтанка https://github.com/nedokormysh/DP_NLP_bootcamp/tree/parsing .

Достаточно очевидно, что этого сайта было недостаточно. И требовалось спарсить лента.ру. Вариант парсинга ленты нашёл на kaggle. https://github.com/nedokormysh/kaggle_nlp2.0/blob/main/notebooks/parsing_news_from_lenta_ru_.ipynb
Насколько помню, то изначально парсил с 2020-01-01 по 2023-10-21.

Было около 26000 записей с фонтанки и около 30000 записей с ленты. Там были дубли и разные хвосты. Поэтому потом привожу код ниже - обработал их для сборки в единый датасет.
В принципе и всё. После этого просто обучил модель catboost, в качестве текстовых признаков указал признак content. Получил результат и заслал на платформу. Это дало отличный результат. Модель сохранил.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Сборка общего датасета.

### Fontanka

Здесь просто загружал данные, которые спарсил изначально

In [ ]:
df_fontanka = pd.DataFrame()

for i in range(1, 13):
    df_ = pd.read_csv(f'https://raw.githubusercontent.com/nedokormysh/DP_NLP_bootcamp/parsing/data/fontanka_full_{i}.csv')
    df_fontanka = pd.concat([df_fontanka, df_])

In [ ]:
# Удаляем дубли
df_fontanka.drop_duplicates(inplace=True)
# Удаляем пропуски
df_fontanka.dropna(inplace=True)
# Переводим тэги к нижнему регистру
df_fontanka['topic']= df_fontanka['topic'].apply(lambda x: x.lower())

df_fontanka.shape

In [ ]:
df_fontanka = df_fontanka[df_fontanka['topic'].isin(['общество',
                                                    #  'город',
                                                    #  'власть',
                                                    #  'политика',
                                                     'финансы',
                                                    #  'новости компаний',
                                                     'спорт',
                                                     'строительство',
                                                     'недвижимость',
                                                     'туризм',
                                                     'технологии',
                                                    #  'образ жизни'
                                                     ])]

In [ ]:
di_fontanka = {'общество' : 0,
               'финансы' : 1,
               'спорт' : 4,
               'строительство' : 6,
               'недвижимость': 6,
               'туризм' : 7,
               'технологии' : 8}

In [ ]:
drop_lst_f = ["url", "datetime", "views", 'title']

df_fontanka.drop(drop_lst_f, axis=1, inplace=True)

### Lenta

Изначально не пользовался гугл диском, а загружал в колаб напрямую.

In [ ]:
df_tbl = pd.read_excel('/content/drive/MyDrive/competitions/NLP2.0/data/parsing_data/lenta2019-01-01_2023-12-21.xlsx')
df_tbl.head()

Здесь просто просмотрел разные статьи с ленты, чтобы верно указать код топика.

In [ ]:
# # category = {1: "общество",
#                  2: "мир",
#                  3: "former_ussr",
#                  4: "economy",
#                  5: 'наука и техника',
#                  6: 'культура',
#                  7: 'интернет и сми',
#                  8: 'спорт',
#                  9: 'из жизни',
#                  12: 'среда обитания',
#                  37: 'силовые структуры',
#                  40: 'бизнес',
#                  47: 'ценности',
#                  48: 'путешествия',
#                  49:'69-парралелль',
#                  53:'нацпроекты',
#                  86:'моя страна',
#                  49:'забота о себе'}

In [ ]:
di_lenta_tmp= {1: 'общество',
               3: 'бывший_СССР',
               4: 'экономика',
               5: 'технологии',
               8: 'спорт',
               37: 'силовые_структуры',
               48: 'путешествия',
               49: 'забота_о_себе'}

In [ ]:
df_tbl['topic'] = pd.Series(df_tbl.bloc.map(di_lenta_tmp))

In [ ]:
df_lenta = df_tbl[df_tbl['topic'].isin(['общество',
                                        'бывший_СССР',
                                        'экономика',
                                        'силовые_структуры',
                                        'технологии',
                                        'спорт',
                                        'путешествия',
                                        'забота_о_себе'])]

In [ ]:
drop_lst = ["docid", "modified", "lastmodtime", "type", "domain", "status",
            "part", "tags", "image_url", "rightcol", "bloc", "pubdate", "url",
            "Unnamed: 0", "snippet",
            'title']

df_lenta.drop(drop_lst, axis=1, inplace=True)

In [ ]:
di_lenta= {'общество': 0,
           'экономика': 1,
           'силовые_структуры': 2,
           'бывший_СССР': 3,
           'спорт': 4,
           'забота_о_себе': 5,
           'путешествия': 7,
           'технологии': 8}

In [ ]:
df_lenta['topic'] = pd.Series(df_lenta.topic.map(di_lenta))

In [ ]:
df_lenta.rename(columns={"text": "content"}, inplace=True)

In [ ]:
df_all = pd.concat([df_fontanka,
#                     # df_ria,
                    df_lenta])

In [ ]:
df_all.to_csv('df_5_fontanka_lenta.csv', index=False)

## 2

Изначально показалось, что необходимо добавить статьи РИА. Это был второй подход к задаче. Я очень много времени потратил на сбор данных с риа. Множество раз не получилось их собрать. Почему-то я верил, что можно запустить один раз и долго подождать, а не собирать кусками данные. (код ноутбука здесь https://github.com/nedokormysh/kaggle_nlp2.0/blob/main/notebooks/Копия_блокнота__ria_parsing_task_ipynb_.ipynb) Это оказалось ошибочной стратегией. В конечном счёте, оказался работать в колабе и на основе приведённого ноутбука на локальном компе добавил пару строк, чтобы скачивать данные через каждые 100 итераций. Это позволило собрать данные. Впрочем, это только всё ухудшило. И видимо данные с риа новостей собирать не следовало. Но было уже очень обидно их не использовать.

### РИА

In [ ]:
df_ria = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/tmp_data/df_ria.csv')
df_ria_2 = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/tmp_data/df_ria_2.csv')
df_ria2 = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/tmp_data/df_ria2.csv')
df_ria_0 = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/tmp_data/df_ria_0.csv')

In [ ]:
df_ria = pd.concat([df_ria, df_ria_2, df_ria2, df_ria_0])

In [ ]:
df_ria.drop_duplicates(inplace=True)

In [ ]:
drop_lst = ["id", "url", "title", "subtitle", "datetime"]

df_ria.drop(drop_lst, axis=1, inplace=True)

In [ ]:
di_ria = {'society': 0,
          'economy': 1,
          'science': 8}

In [ ]:
df_ria['topic'] = pd.Series(df_ria.topic.map(di_ria))

## 3
Собственно изначальная модель насколько помню была просто обучена даже не на всём объёме данных, а только на X_train.

## Получение разбиения.

In [ ]:
# загрузка данных
trainw2v = pd.read_csv(paths.path_prep_train_no_ria)
test = pd.read_csv(paths.path_prep_test)

In [ ]:
# Данные оптимизировал по объёму


# def reduce_mem_usage(df):
#     """ iterate through all the columns of a dataframe and modify the data type
#         to reduce memory usage.
#     """
#     start_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

#     for col in df.columns:
#         col_type = df[col].dtype

#         if col_type != object:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)
#         # else:
#         #     df[col] = df[col].astype('category')

#     end_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
#     print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

#     return df

# def import_data(file):
#     """create a dataframe and optimize its memory usage"""
#     df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
#     df = reduce_mem_usage(df)
#     return df


# print('train')
# train = import_data(paths.path_train)

# print('-' * 80)
# print('test')
# test = import_data(paths.path_test)

In [ ]:
# Удаляем дубли
trainw2v.drop_duplicates(inplace=True)
# Удаляем пропуски
trainw2v.dropna(inplace=True)

print(f'train shape = {trainw2v.shape},  test shape = {test.shape}')
print(f'train, test is null: {trainw2v.isna().any().any(), test.isna().any().any()}')

In [ ]:
text_features = ['content',
                #  'content_clean' в исходных данных колонка content_clean появилась позже
                 ]
# cat_features_indxs = [1]
targets = ['topic']
# cat_cols = ['content', 'content_clean']

filtered_features = [i for i in trainw2v.columns if (i not in targets)]

In [ ]:
X = trainw2v[filtered_features].drop(targets, axis=1, errors="ignore")
y = trainw2v["topic"]

In [ ]:
# Разбиение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RND_STATE)

Собственно на этом X_train и была построена первая модель.

## 4

Одновременно с загрузкой риа, пытался при помощи optuna подобрать гиперпараметры для базовой модели.

## Optuna для catboost

In [ ]:
def fit_catboost(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {
        'iterations' : 500,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.04, step=0.001),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 30, 70, step=10),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.3, 0.4, step=0.1),
        "auto_class_weights": trial.suggest_categorical("auto_class_weights",
                                                        [#"SqrtBalanced",
                                                         #"Balanced",
                                                         "None"]),
        "depth": trial.suggest_int("depth", 9, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", [#"Ordered",
                                                                     "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type",
                                                              [#"Bayesian",
                                                               "Bernoulli",
                                                               #"MVS"
                                                               ]),
        "used_ram_limit": "10gb",
        "eval_metric": "Accuracy", # Тоже стоит заранее определиться
        # 'loss_function': 'MultiClass'
    }


    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)

    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)


    clf = CatBoostClassifier(
        **param,
        thread_count=-1,
        random_seed=RND_STATE,
        text_features=text_features)
    #  Создаем объект callback
    pruning_callback = CatBoostPruningCallback(trial, "Accuracy")

    clf.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        verbose=0,
        plot=False,
        early_stopping_rounds=5,
        callbacks=[pruning_callback],
    )

    # запускаем процесс прунинга
    pruning_callback.check_pruned()

    y_pred = clf.predict(X_val)
    return clf, y_pred

In [ ]:
def objective(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

    scores_simple_acc, scores_balanced_acc, models = [], [], []

    for train_idx, valid_idx in kf.split(X_train):
        train_data = X_train.iloc[train_idx, :], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx, :], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_catboost(trial, train_data, valid_data) # Определили выше

        # scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
        scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break

    result = np.mean(scores_balanced_acc)
    # result = scores_balanced_acc
    print(f'result = {result}')

    if return_models:
        return result, models
    else:
        return result

In [ ]:
from optuna.integration import CatBoostPruningCallback, LightGBMPruningCallback, XGBoostPruningCallback
from optuna.integration.pytorch_lightning import PyTorchLightningPruningCallback
optuna.logging.set_verbosity(optuna.logging.WARNING)
# optuna.logging.set_verbosity(optuna.logging.ERROR)

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            study_name="Catboost_clf_simple",
                            sampler=sampler)
study.optimize(objective,
               n_trials=40,
               n_jobs=-1,
               show_progress_bar=True)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# История изменения от числа испытаний
optuna.visualization.plot_optimization_history(study)

In [ ]:
params = ['learning_rate',
          'l2_leaf_reg',
          'colsample_bylevel',
          'auto_class_weights',
          'depth',
          'bootstrap_type',
          'boosting_type']

optuna.visualization.plot_slice(study, params=params, target_name='accuracy')

Это заняло огромное количество времени. Вообще optuna для вот базовой модели делал только на части X_train. Брал какое-то количество записей. И это оказалось абсолютно бесполезно. Ничем не улучшило качество.

In [ ]:
# В дальнейшем, потребовалось в разные модели подавать разные колонки, поэтому был написан миникласс

class CustomSelectColumns():
    '''
    .. notes::
    Класс для вытаскивания из датасета только требуемых колонок
    '''
    def __init__(self,
                 fil_feats: list[str]=None, clean: bool=False):
        self.fil_feats = fil_feats
        self.clean = clean

        # print('Init done')
    def fit(self, X: pd.DataFrame, y: pd.Series=None):
        return self

    def transform(self, X: pd.DataFrame, y: pd.Series=None) -> pd.DataFrame:
        X_ = X.copy()
        X_filtered = X_[self.fil_feats]
        X_filtered = pd.DataFrame(X_filtered)
        if self.clean:
            return X_filtered[self.fil_feats[0]].str.split()
        else:
            return pd.DataFrame(X_filtered)

In [ ]:
# более старый вариант выбора колонок, здесь можно возвратить именно строки, а не просто датасет

# class CustomColumns():
#     '''
#     .. notes::
#     Класс для вытаскивания из датасета только требуемых колонок
#     '''
#     def __init__(self,
#                  fil_feats: list[str]=None, clean: bool=False):
#         # self.fil_feats = fil_feats
#         self.clean = clean

#         # print('Init done')
#     def fit(self, X: pd.DataFrame, y: pd.Series=None):
#         return self

#     def transform(self, X: pd.DataFrame, y: pd.Series=None) -> pd.DataFrame:
#         X_ = X.copy()
#         X_filtered = X_[self.fil_feats]
#         X_filtered = pd.DataFrame(X_filtered)
#         if self.clean:
#             return X_filtered[self.fil_feats[0]].str.split()
#         else:
#             return pd.DataFrame(X_filtered)

In [ ]:
Catboost_pipe = Pipeline([
    ('col_selector', CustomSelectColumns(['content'])),
    ('Catboost_simple', CatBoostClassifier(eval_metric='Accuracy',
                           iterations=200,
                           thread_count=-1,
                           random_seed=RND_STATE,
                           text_features=text_features))
    ])

# train_dataset = Pool(data=X_train, label=y_train, text_features=['content'])
# eval_dataset = Pool(data=X_val, label=y_val, text_features=['content'])

Catboost_pipe.fit(X_train, y_train,
                  Catboost_simple__text_features=['content'],
                  Catboost_simple__verbose=5,
                  Catboost_simple__early_stopping_rounds=10,
                  Catboost_simple__eval_set=(X_val[['content']], y_val)
                  )

In [ ]:
# Это самый первый вариант запуска и построения модели. Здесь не используются pipeline.
# Зато используются Pool библиотеки catboost для ускорения работы

# %%time
# clf_1 = CatBoostClassifier(eval_metric= 'Accuracy',
#                            iterations=200,
#                            thread_count=-1,
#                            random_seed=RND_STATE,
#                            text_features=text_features)

# train_dataset = Pool(data=X_train, label=y_train, text_features=['content'])
# eval_dataset = Pool(data=X_val, label=y_val, text_features=['content'])

# clf_1.fit(train_dataset,
#         eval_set=eval_dataset,
#         verbose=50,
#         early_stopping_rounds=200)

## 5
Одновременно шла работа для создания данных для других моделей. Для этого использовались навыки, полученные в ходе обучения. И была создана доп. колонка content_clean. при помощи библиотеки natasha

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [ ]:
stop_words = stopwords.words('russian')
stop_words.extend(['что', 'это', 'так',
                    'вот', 'быть', 'как',
                    'в', '—', 'к', 'за', 'из', 'из-за',
                    'на', 'ок', 'кстати',
                    'который', 'мочь', 'весь',
                    'еще', 'также', 'свой',
                    'ещё', 'самый', 'ул', 'комментарий',
                    'английский', 'язык'])

In [ ]:
# функция для обработки текстов

def text_prep(text) -> str:
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    lemmas = [_.lemma for _ in doc.tokens]
    words = [lemma for lemma in lemmas if lemma.isalpha() and len(lemma) > 2]
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

In [ ]:
# # Удаляем дубли
# train.drop_duplicates(inplace=True)
# # Удаляем пропуски
# train.dropna(inplace=True)

# print(f'train shape = {train.shape},  test shape = {test.shape}')
# print(f'train, test is null: {train.isna().any().any(), test.isna().any().any()}')

In [ ]:
%%time
train['content_clean'] = train.content.apply(text_prep)

In [ ]:
# train.to_csv('train_02.01_prep.csv', index=False)

In [ ]:
%%time
test['content_clean'] = test.content.apply(text_prep)

Соответственно в дальнейшем в данных уже всегда присутствовала эта колонка.

Хотя запускалось всё множесто раз. Потому что после сборки данных с риа и получения плохих результатов, было решено увеличить количество новостей с лента.ру. И несколько раз изначальный датасет пересобирался, и, следовательно, и все шаги по обработке данных проводились заново.

## 6
Векторизация слов. Изначально для других моделей выбрал стандартную word2vec. В принципе, долгое время думал, что нужно стараться быть более близким к пройдённому на курсе.
Это сыграло плохую шутку - я пропустил последнюю лекцию, где показывались предобученные модели. И слишком долго бился только в стандартные методы.

Для векторизации слов использовал word2vec. fasttext - не давал никаких улучшений. поэтому от него отказался.

Для вектора предложений использовал tfidf. Впрочем потом и обычное усреднение тоже было использовано. Всё это было напрасно.

## TFIDF

In [ ]:
# Для получения вектора предложения используем tf_idf
class TfidfEmbeddingVectorizer(object):
    """Get tfidf weighted vectors"""
    def __init__(self, model):
        self.word2vec = model.wv
        self.word2weight = None
        self.dim = model.vector_size

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec.get_vector(w) * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

## Word2vec

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X.content_clean.str.split(),
#                                                   y,#.values,
#                                                   test_size=0.2,
#                                                   random_state=RND_STATE)


model_w2v = Word2Vec(sentences=X_train,
                 vector_size=300,
                 min_count=5,
                 window=25,
                 seed=RND_STATE)

In [ ]:
with open(f'model_w2v.pkl', "wb") as f:
        pickle.dump(model_w2v, f)

In [ ]:
with open(f"/content/drive/MyDrive/competitions/NLP2.0/models/model_w2v.pkl", "rb") as f:
        model_w2v = pickle.load(f)

# Simple models

## log reg

Соответственно в первом варианте, построения логистической регрессии даже был немного другой класс для выбора колонок. И там возвращались именно строки. Которые подавались по pipeline в tfidf и scaler.

В дальнейшем создал отдельные колонки в датасетах, в которых были усреднённые значения векторов, tfidf значения. И уже и построения моделей было немного другим.

In [ ]:
# pipeline variant
%%time
pipe_log = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                 ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))])

pipe_log.fit(X_train, y_train)

print(classification_report(y_val, pipe_log.predict(X_val)))

In [ ]:
def fit_log_reg(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {"dual": False,
             "C": trial.suggest_float("C", 2.5, 3, step=0.1),
             "solver": trial.suggest_categorical("solver",
                                                 ["sag",
                                                  #"saga",
                                                  "lbfgs"
                                                  ]),
             "random_state": RND_STATE,
             "max_iter": trial.suggest_int("max_iter", 5500, 6300, step=200),
             'multi_class': trial.suggest_categorical("multi_class",
                                                      [#'auto',
                                                       'ovr',
                                                       #'multinomial'
                                                       ]),
             'n_jobs': -1,
             'verbose': 0
    }

    # print(f'param["solver"] = {param["solver"]}')
    if param["solver"] == "saga":
        param["penalty"] = trial.suggest_categorical("penalty",
                                             ['l1',
                                              'l2', #'elasticnet',
                                              None])
    else:
         param["penalty"] = trial.suggest_categorical("penalty",
                                             ['l2',
                                              #None
                                              ])

    clf = LogisticRegression(**param)

    pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(model_w2v)),
                    ('scaler', StandardScaler()),
                    ('clf', clf)])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_val)
    return pipe, y_pred

In [ ]:
def objective_log(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

    scores_simple_acc, scores_balanced_acc, models = [], [], []

    for train_idx, valid_idx in kf.split(X_train.index):
        train_data = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_log_reg(trial, train_data, valid_data)

        # scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
        scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break

    result = np.mean(scores_balanced_acc)
    # print(f'result = {result}')

    if return_models:
        return result, models
    else:
        return result

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            sampler=sampler,
                            study_name="Log_Reg_clf")
study.optimize(objective_log,
               n_trials=10,
               n_jobs=-1,
               show_progress_bar=True)

Все модели оптимизировал optuna - всё бесполезно.

## SVM

In [ ]:
#pipeline version
%%time
pipe_svc = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                     ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                     ('scaler', StandardScaler()),
                     ('clf', SVC(random_state=RND_STATE,
                             max_iter=10000,
                            #  class_weight='balanced'
                             ))])

pipe_svc.fit(X_train, y_train)

print(classification_report(y_val, pipe_svc.predict(X_val)))

# Stack

Соответственно, когда обучил 3 модели: catboost, svm, log reg - сделал стекинг моделей. И это, в принципе дало мне наилучший результат. Впрочем, я его не использовал. И всё равно это давало улучшения на доли процентов. Причём это улучшение было именно уже на private leaderboard/

In [ ]:
# Некоторые модели для построение ансамбля
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier

In [ ]:
# список базовых моделей
estimators = [("CatBoost", Catboost_pipe),
              ('SVC', pipe_svc),
              ('Log_Reg', pipe_log)]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False, max_iter=200),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier = meta_model
stacking_classifier

In [ ]:
preds_st = stacking_classifier.predict(X_val)
balanced_accuracy_score(y_true=y_val, y_pred=preds_st)
accuracy_score(y_true=y_val, y_pred=preds_st)

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = stacking_classifier.predict(test)
df_sub['topic'] = preds
df_sub.to_csv('try_5.csv', index=False)

# 7
После неудачи со стекингом, попробовал обучить ещё один бустинг.

In [ ]:
%%time
XGboost_pipe = Pipeline([('col_selector', CustomSelectColumns(['content_clean'], clean=True)),
                     ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                    #  ('scaler', StandardScaler()),
                     ('clf', XGBClassifier(random_state=RND_STATE,
                            #  max_iter=10000,
                            #  class_weight='balanced'
                             ))])

XGboost_pipe.fit(X_train, y_train,
                #  clf__eval_set=(X_val, y_val)
                 )

print(classification_report(y_val, XGboost_pipe.predict(X_val)))

опять же optuna использовалась. И одновременно добавлялись и добавлялись разные варианты данных.

Где-то в этот момент я вспомнил, что в обсуждениях был вопрос о предобученных моделях. Но вообще я ответ понял, что можно использовать, но вообще было бы лучше на стандартных моделях работать.

Использования catbosst + text_features - я вообще тоже считал неким читерством. И пытался выжать скорее из sklearn моделей что-то хорошее.

+ важным нюансом оказалось, что я вообще пропустил, что была последняя лекция, на которой показывались варианты работы с bert.
Т.е. когда в обсуждениях возник вопрос о предобученных моделях, я думал, что их вообще на курсе не показывали. И люди, просто с большим количеством изначальных знаний, которые были получены не на курсе, могли пользоваться своими знаниями.

В общем, всё это было огромной ошибкой.

Обучал ещё несколько разных вариантов моделей из sklearn. multinomialNB, sgdclassifier ... Использовал стекинг. Всё это не давало результатов. + время конкурса уже закончилось. 4 числа.

# 8

Где-то пятого числа я уже окончательно понял, что мне ничего не выжать из простых моделей. И вспомнил про предобученные модели. Я, в принципе, не очень знал, что это. Подумал, что видимо векторизаторы. И нашёл первый попавшийся

## navec

In [ ]:
!pip install navec

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

In [ ]:
from navec import Navec

path = '/content/navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
# Разбиение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RND_STATE)

In [ ]:
def get_mean_navec_vector(sentence):
    Sum = 0
    Count = 0

    words = sentence.split()

    for w in words:
        if w in navec:
            Sum += navec[w]
            Count += 1

    return Sum / Count

In [ ]:
NewCols = ['col'+str(i) for i in range(300)]

In [ ]:
X_train_navec = X_train.copy()
X_test_navec = X_val.copy()

X_train_navec['vectors'] = X_train['content_clean'].map(get_mean_navec_vector)
X_test_navec['vectors'] = X_val['content_clean'].map(get_mean_navec_vector)

In [ ]:
X_train_navec[NewCols] = pd.DataFrame(X_train_navec['vectors'].tolist(), index= X_train_navec.index)
X_test_navec[NewCols] = pd.DataFrame(X_test_navec['vectors'].tolist(), index= X_test_navec.index)

In [ ]:
X_train_navec.drop(['content','content_clean','vectors'], axis=1, inplace=True)
X_test_navec.drop(['content','content_clean', 'vectors'], axis=1, inplace=True)

Добавление просто этих дополнительных векторов в датасет только ухудшило ситуацию. Но я не получал окончательное вариант усреднения для предложений, а просто подавал вот полученные вектора (length = 300) в модель.

Я подумал, что надо попоробовать векторизаторы из !pip install transformers -q. И даже запустил векторизацию, впрочем всё равно это было не совсем так как было показано на занятии. Одним из важных отличий -подавал не батчами. И я около 5 часов ждал векторизации хотя бы данных train, но не дождался и выключил.

Уже оставалось очень мало времени, как я предполагал. И ещё верил, что может быть с navec векторами и стекингом у меня получится выжать что-то побольше. + считал, что использовать предобученные векторизаторы скорее не приветствуется.

Поэтому окончательный вариант был большое количесто стекинга https://github.com/nedokormysh/kaggle_nlp2.0/blob/main/notebooks/NLP2_classification_full_all_new_last.ipynb

In [ ]:
# список базовых моделей
estimators = [("CatBoost", Catboost_pipe),
              ('SVC', pipe_svc),
              ('XGboost_pipe', XGboost_pipe),
              ('pipe_SGD', pipe_SGD),
              ('Log_Reg', pipe_log)]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False, max_iter=200),
    # n_jobs=-1,
    verbose=True,
)

stacking_classifier = meta_model
stacking_classifier

Пытался сблендить результаты разных моделей -выбрать самый часто встречающийся ответ - не дало результатов.

В окончательное решение попала первая модель с хорошим скором catboost на text_features и стекинг c navec векторами. (стекинг по локальному скору и по скору на public был достаточно плох, но была надежда, что возможно некая генерализация решения будет - опять ошибочный вариант.)

Собственно, зная, что соревнование окончится 8-го - можно было бы ещё раз попробовать обучить catboost, но уже на bert векторах. Но я подумал, что просто уже не успею ничего сделать. Немного увеличение сроков для меня играло скорее в минус. Т.е. для меня это было скорее неожиданностью.

Лучшим решением остался catboost, обученный только на части данных.

Насколько я понял, то топовые решения использовали bert. И я увидел, что bert показывался на занятиях только 6-го числа.